In [1]:
import pandas as pd

# Data files path
file1 = "../data/dft-road-casualty-statistics-casualty-2022.csv"
file2 = "../data/dft-road-casualty-statistics-collision-2022.csv"
file3 = "../data/dft-road-casualty-statistics-vehicle-2022.csv"

# Reading CSV files into DataFrames
df_casualty = pd.read_csv(file1)
df_collision = pd.read_csv(file2)
df_vehicle = pd.read_csv(file3)

C:\Users\ATAUMAR\AppData\Local\Temp\ipykernel_2504\1912417982.py:9: DtypeWarning: Columns (0,2) have mixed types. Specify dtype option on import or set low_memory=False.
  df_casualty = pd.read_csv(file1)
C:\Users\ATAUMAR\AppData\Local\Temp\ipykernel_2504\1912417982.py:10: DtypeWarning: Columns (0,2) have mixed types. Specify dtype option on import or set low_memory=False.
  df_collision = pd.read_csv(file2)
C:\Users\ATAUMAR\AppData\Local\Temp\ipykernel_2504\1912417982.py:11: DtypeWarning: Columns (0,2) have mixed types. Specify dtype option on import or set low_memory=False.
  df_vehicle = pd.read_csv(file3)


In [2]:
# Printing column names of each DataFrame
print("Column names of 'dft-road-casualty-statistics-casualty-2022.csv':")
print(df_casualty.columns)
print("\nColumn names of 'dft-road-casualty-statistics-collision-2022.csv':")
print(df_collision.columns)
print("\nColumn names of 'dft-road-casualty-statistics-vehicle-2022.csv':")
print(df_vehicle.columns)

Column names of 'dft-road-casualty-statistics-casualty-2022.csv':
Index(['accident_index', 'accident_year', 'accident_reference',
       'vehicle_reference', 'casualty_reference', 'casualty_class',
       'sex_of_casualty', 'age_of_casualty', 'age_band_of_casualty',
       'casualty_severity', 'pedestrian_location', 'pedestrian_movement',
       'car_passenger', 'bus_or_coach_passenger',
       'pedestrian_road_maintenance_worker', 'casualty_type',
       'casualty_home_area_type', 'casualty_imd_decile', 'lsoa_of_casualty'],
      dtype='object')

Column names of 'dft-road-casualty-statistics-collision-2022.csv':
Index(['accident_index', 'accident_year', 'accident_reference',
       'location_easting_osgr', 'location_northing_osgr', 'longitude',
       'latitude', 'police_force', 'accident_severity', 'number_of_vehicles',
       'number_of_casualties', 'date', 'day_of_week', 'time',
       'local_authority_district', 'local_authority_ons_district',
       'local_authority_highway', 'fi

In [ ]:
# Merging datasets on 'accident_index'
df_merged = pd.merge(df_casualty, df_collision, on='accident_index')
df_merged = pd.merge(df_merged, df_vehicle, on='accident_index')

print("Merged DataFrame:")
df_merged.head()